In [1]:
import os
from typing import Callable

from edgedroid.models.sampling import *
from edgedroid.models.timings import *

rtts = np.linspace(0, 5, 10)[1:]

runs_per_model = 15
task_steps = 100

timing_models: Dict[str, Callable[[], ExecutionTimeModel]] = {
    "empirical-low": lambda: EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0),
    "empirical-high": lambda: EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0),
    "theoretical-low": lambda: TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0),
    "theoretical-high": lambda: TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0),
    # "constant": lambda: ConstantExecutionTimeModel.from_default_data(),
    # "naive": lambda: NaiveExecutionTimeModel.from_default_data(),
    # "fitted-naive": lambda: FittedNaiveExecutionTimeModel.from_default_data(),
}

In [2]:
import edgedroid.data as e_data
data, *_ = e_data.load_default_exec_time_data()
data

,run_id,ttf,exec_time,neuroticism
0,134146,0.597441,3.654797,0.375
1,134146,0.553513,4.438645,0.375
2,134146,0.561716,2.943222,0.375
3,134146,0.586512,5.405761,0.375
4,134146,0.558940,5.225161,0.375
...,...,...,...,...
6755,137353,0.557074,6.439071,0.625
6756,137353,0.534339,4.680858,0.625
6757,137353,0.560288,3.467878,0.625
6758,137353,0.579000,2.325759,0.625


In [3]:
exec_time_qs = np.round(data["exec_time"].describe(percentiles=[0.25, 0.5, 0.75])[["25%", "50%", "75%"]], decimals=1)
exec_time_qs

25%    3.9
50%    5.2
75%    7.0
Name: exec_time, dtype: float64

In [4]:
from edgedroid.models.sampling.adaptive import _aperiodic_instant_iterator
from typing import NamedTuple

class SamplingResult(NamedTuple):
    final_sampling_instant: float
    num_samples: int

class ExpSampling(BaseFrameSamplingModel, abc.ABC):
    @abc.abstractmethod
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        pass


class GreedySampling(ZeroWaitFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        num_samples = 1
        instant = 0.0

        while instant <= target_exec_time:
            instant += rtt
            num_samples += 1

        return SamplingResult(instant, num_samples)

class IdealSampling(IdealFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        return SamplingResult(target_exec_time, 1)

class PeriodicSampling(RegularFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        num_samples = 1
        instant = self._interval

        interval = max(self._interval, rtt)

        while instant <= target_exec_time:
            instant += interval
            num_samples += 1

        return SamplingResult(instant, num_samples)

class HoldSampling(HoldFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        num_samples = 1
        instant = self._hold_time

        while instant <= target_exec_time:
            instant += rtt
            num_samples += 1

        return SamplingResult(instant, num_samples)

class AdaptiveSamplingMixin(BaseAperiodicFrameSamplingModel, ExpSampling, abc.ABC):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        self._timing_model.advance(prev_ttf)
        alpha = self.get_alpha()
        beta = self.get_beta()

        rtts = deque()
        prev_instant = 0.0
        for instant in _aperiodic_instant_iterator(
            mu=self._timing_model.get_expected_execution_time(),
            alpha=alpha,
            beta=beta,
        ):
            rtts.append(rtt)
            instant = max(instant, prev_instant + rtt)

            if instant > target_exec_time:
                self.record_rtts(rtts)
                return SamplingResult(instant, len(rtts))
            else:
                prev_instant = instant

class AdaptiveSampling(AperiodicFrameSamplingModel, AdaptiveSamplingMixin):
    pass

class AdaptivePowerSampling(AperiodicPowerFrameSamplingModel, AdaptiveSamplingMixin):
    pass

sampling_schemes: Dict[str, Callable[[], ExpSampling]] = {
    "greedy": lambda : GreedySampling.from_default_data(),
    "ideal": lambda : IdealSampling.from_default_data(),
}

sampling_schemes.update({
    "adaptive-empirical": lambda : AdaptiveSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=None)),
    "adaptive-empirical-low": lambda : AdaptiveSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    "adaptive-empirical-high": lambda : AdaptiveSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-theoretical": lambda : AdaptiveSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=None)),
    "adaptive-theoretical-low": lambda : AdaptiveSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    "adaptive-theoretical-high": lambda : AdaptiveSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-fitted-naive": lambda : AdaptiveSampling.from_default_data(FittedNaiveExecutionTimeModel.from_default_data())
})

sampling_schemes.update({
    f"adaptive-constant-Q{i + 1}-{t:0.1f}s":
        lambda : AdaptiveSampling.from_default_data(ConstantExecutionTimeModel(float(t))) for i, t in enumerate(exec_time_qs)
})

sampling_schemes.update({
    "adaptive-power-empirical": lambda : AdaptivePowerSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=None)),
    "adaptive-power-empirical-low": lambda : AdaptivePowerSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    "adaptive-power-empirical-high": lambda : AdaptivePowerSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-power-theoretical": lambda : AdaptivePowerSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=None)),
    "adaptive-power-theoretical-low": lambda : AdaptivePowerSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    "adaptive-power-theoretical-high": lambda : AdaptivePowerSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-power-fitted-naive": lambda : AdaptivePowerSampling.from_default_data(FittedNaiveExecutionTimeModel.from_default_data())
})

sampling_schemes.update({
    f"adaptive-power-constant-Q{i + 1}-{t:0.1f}s":
        lambda : AdaptivePowerSampling.from_default_data(ConstantExecutionTimeModel(float(t))) for i, t in enumerate(exec_time_qs)
})

sampling_schemes.update({
    f"periodic-{t:0.1f}s": lambda : PeriodicSampling.from_default_data(sampling_interval_seconds=float(t)) for t in (1, 3, 5)
})
sampling_schemes.update({
    f"hold-{t:0.1f}s": lambda : HoldSampling.from_default_data(hold_time_seconds=float(t)) for t in (1, 3, 5)
})

list(sampling_schemes.keys())

['greedy',
 'ideal',
 'adaptive-empirical',
 'adaptive-empirical-low',
 'adaptive-empirical-high',
 'adaptive-theoretical',
 'adaptive-theoretical-low',
 'adaptive-theoretical-high',
 'adaptive-fitted-naive',
 'adaptive-constant-Q1-3.9s',
 'adaptive-constant-Q2-5.2s',
 'adaptive-constant-Q3-7.0s',
 'adaptive-power-empirical',
 'adaptive-power-empirical-low',
 'adaptive-power-empirical-high',
 'adaptive-power-theoretical',
 'adaptive-power-theoretical-low',
 'adaptive-power-theoretical-high',
 'adaptive-power-fitted-naive',
 'adaptive-power-constant-Q1-3.9s',
 'adaptive-power-constant-Q2-5.2s',
 'adaptive-power-constant-Q3-7.0s',
 'periodic-1.0s',
 'periodic-3.0s',
 'periodic-5.0s',
 'hold-1.0s',
 'hold-3.0s',
 'hold-5.0s']

In [5]:
power_mw = {
    "comm": 0.045,
    "idle": 0.015
}  # Watts

proc_time = 0.3 # 300 ms

def run_combination(timing: str, sampling: str, rtt: float, repetition: int) -> pd.DataFrame:
    timing_model = timing_models[timing]()
    sampling_model = sampling_schemes[sampling]()
    prev_ttf = rtt
    cumulative_duration = 0.0
    cumulative_samples = 0

    # for power calculations:
    comm_time_per_sample = rtt - proc_time
    cumulative_energy = 0.0

    rows = deque()

    for step in range(1, task_steps + 1):
        exec_time = timing_model.advance(prev_ttf).get_execution_time()
        final_sample, num_samples = sampling_model.constant_rtt_sampling(rtt, prev_ttf, exec_time)

        duration = final_sample + rtt
        cumulative_duration += duration
        cumulative_samples += num_samples
        ttf = duration - exec_time
        wait_time = ttf - rtt

        # calculate power
        total_comm_time = comm_time_per_sample * num_samples
        total_idle_time = duration - total_comm_time

        total_energy = (total_idle_time * power_mw["idle"]) + (total_comm_time * power_mw["comm"])
        cumulative_energy += total_energy

        rows.append(
            {
                "timing_model": timing,
                "sampling_scheme": sampling,
                "rtt": rtt,
                "step": step,
                "previous_ttf": prev_ttf,
                "execution_time": exec_time,
                "step_duration": duration,
                "ttf": ttf,
                "wait_time": wait_time,
                "samples": num_samples,
                "cumulative_duration": cumulative_duration,
                "cumulative_samples": cumulative_samples,
                "repetition": repetition,
                "energy": total_energy,
                "cumulative_energy": cumulative_energy,
            }
        )
        prev_ttf = ttf

    return pd.DataFrame(rows)

In [6]:
from typing import Tuple
from tqdm.notebook import tqdm
import multiprocess as mp
import shutil

result_path = "./sampling_scaling_rtt.gzip"
combs = set(itertools.product(timing_models.keys(), sampling_schemes.keys(), rtts, range(1, runs_per_model + 1)))

# only calculate missing results
results = deque()
try:
    old_results = pd.read_parquet(result_path)
    existing_combinations = set(old_results[["timing_model", "sampling_scheme", "rtt", "repetition"]].itertuples(index=False))
    shutil.rmtree(result_path)
    results.append(old_results)
except FileNotFoundError:
    existing_combinations = set()

combs.difference_update(existing_combinations)
if len(combs) == 0:
    print("No missing combinations.")
else:
    with tqdm(total=len(combs), bar_format="{l_bar}{bar}{n_fmt}/{total_fmt} [Time: {elapsed}]") as bar, mp.Pool(os.cpu_count() - 2) as pool:
        bar.set_description("Running timing model/sampling scheme combinations...")

        def make_callback(cmb: Tuple[str, str, float]) -> Callable[[...], None]:
            def _cb(*args, **kwargs) -> None:
                bar.update()
                bar.set_description(f"Running timing model/sampling scheme combinations...")

            return _cb

        procs = [
            pool.apply_async(run_combination, args=c, callback=make_callback(c)) for c in combs
        ]

        for p in procs:
            results.append(p.get())

results = pd.concat(results, ignore_index=True)

results["timing_model"] = results["timing_model"].astype(
    pd.CategoricalDtype(["fitted-naive", "empirical-low", "empirical-high", "theoretical-low", "theoretical-high"], ordered=True)
)
results["sampling_scheme"] = results["sampling_scheme"].astype(
    pd.CategoricalDtype(sampling_schemes.keys(), ordered=False)
)
results.to_parquet(result_path, partition_cols=["timing_model", "sampling_scheme"], compression="gzip")
results

  0%|          0/15120 [Time: 00:00]

,timing_model,sampling_scheme,rtt,step,previous_ttf,execution_time,step_duration,ttf,wait_time,samples,cumulative_duration,cumulative_samples,repetition,energy,cumulative_energy
0,empirical-high,adaptive-constant-Q2-5.2s,2.777778,1,2.777778,7.071596,11.111111,4.039515,1.261737,3,11.111111,3,12,0.389667,0.389667
1,empirical-high,adaptive-constant-Q2-5.2s,2.777778,2,4.039515,8.879713,12.234523,3.354810,0.577032,3,23.345634,6,12,0.406518,0.796185
2,empirical-high,adaptive-constant-Q2-5.2s,2.777778,3,3.354810,4.495618,9.832383,5.336765,2.558987,2,33.178017,8,12,0.296152,1.092337
3,empirical-high,adaptive-constant-Q2-5.2s,2.777778,4,5.336765,0.533615,7.452374,6.918759,4.140981,1,40.630391,9,12,0.186119,1.278456
4,empirical-high,adaptive-constant-Q2-5.2s,2.777778,5,6.918759,8.872578,14.013848,5.141270,2.363493,3,54.644239,12,12,0.433208,1.711664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511995,theoretical-low,adaptive-empirical-low,3.888889,96,6.888578,5.993087,10.922816,4.929729,1.040840,1,1123.539389,127,13,0.271509,30.526758
1511996,theoretical-low,adaptive-empirical-low,3.888889,97,4.929729,7.726390,15.054552,7.328162,3.439273,2,1138.593941,129,13,0.441152,30.967909
1511997,theoretical-low,adaptive-empirical-low,3.888889,98,7.328162,4.544998,10.680071,6.135073,2.246184,1,1149.274012,130,13,0.267868,31.235777
1511998,theoretical-low,adaptive-empirical-low,3.888889,99,6.135073,6.428863,10.680071,4.251207,0.362319,1,1159.954083,131,13,0.267868,31.503645
